In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import torch

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from transformers.trainer_utils import EvalPrediction
from sklearn.model_selection import train_test_split
from datetime import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install mlflow
import mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 16.3 MB/s eta 0:00:00


In [3]:
MODEL_NAME = "HuggingFaceH4/zephyr-7b-alpha"
OUTPUT_DIR = "./zephyr-multitask-outputs"
MAX_LENGTH = 512
EPOCHS = 3
BATCH_SIZE = 1
LEARNING_RATE = 5e-5
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("zephyr_multitask_finetune")



2025/06/29 06:36:07 INFO mlflow.tracking.fluent: Experiment with name 'zephyr_multitask_finetune' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/912633963179435852', creation_time=1751178967716, experiment_id='912633963179435852', last_update_time=1751178967716, lifecycle_stage='active', name='zephyr_multitask_finetune', tags={}>

In [6]:
def format_sample(example):
    text = example["messages"][0]["content"]
    target = {
        "intent": example["intent"][0],
        "sentiment": example["sentiment"][0],
        "topic": example["topic"][0],
        "entities": example["entities"]
    }
    formatted = f"<|system|>You are a helpful assistant.<|user|>Classify the following: {text}<|assistant|>{json.dumps(target)}"
    return {"text": formatted}

with open("labeled_2025-06-28.jsonl", "r", encoding="utf-8") as f:
    raw_data = [json.loads(line) for line in f]

formatted_data = [format_sample(item) for item in raw_data]
dataset = Dataset.from_list(formatted_data)




In [7]:
# === Tokenizer and Tokenization ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

tokenized_dataset = dataset.map(tokenize)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Map:   0%|          | 0/5363 [00:00<?, ? examples/s]

In [ ]:
# === Train/test split ===
train_test = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test["train"]
eval_dataset = train_test["test"]

# === Data collator ===
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# === Model ===
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True, torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32)

# === Evaluation ===
def compute_metrics(eval_pred: EvalPrediction):
    return {}


config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs",
    logging_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
with mlflow.start_run(run_name=f"zephyr-multitask-{datetime.now().strftime('%Y%m%d-%H%M%S')}"):
    mlflow.log_param("model", MODEL_NAME)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("max_length", MAX_LENGTH)
    mlflow.log_param("learning_rate", LEARNING_RATE)

    trainer.train()

    # Save model + tokenizer
    trainer.save_model(f"{OUTPUT_DIR}/final")
    tokenizer.save_pretrained(f"{OUTPUT_DIR}/final")


In [ ]:
test_samples = [
    "Why is my internet so slow today again?",
    "Loved the support, thank you!",
    "You charged me extra again, unbelievable!",
    "Can I reschedule my appointment?",
    "App isn't loading, keeps crashing!"
]

model.eval()
for sample in test_samples:
    prompt = (
        f"Text: {sample}\n"
        f"Classify into JSON:\n"
        f"{{\"intent\": , \"sentiment\": , \"topic\": , \"entities\": }}"
    )
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=128)
    print(f"📩 Input: {sample}")
    print(tokenizer.decode(outputs[0], skip_special_tokens=True).split("Classify into JSON:")[-1].strip())
    print("-" * 60)
"""))